In [197]:
import ee
import geemap
import sys

try:
    # Initialize the library.
    ee.Initialize()
    print('Google Earth Engine has initialized successfully!')
except ee.EEException as e:
    print(e)
    print('Google Earth Engine has failed to initialize!')
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise
    
def add_metrics(image):
    ndvi = image.normalizedDifference(['B7', 'B4']).rename('NDVI')
    gndvi = image.normalizedDifference(['B7', 'B3']).rename('GNDVI')   
    savi = ee.Image().expression('((NIR-Red)/(NIR+Red+0.5))*1.5', {
        'NIR': image.select('B8').multiply(0.0001),
        'Red': image.select('B4').multiply(0.0001)
      }).rename('SAVI')
#     cvi = ee.Image().expression('(NIR/Green)*(Red/Green)', {
#         'NIR': input.select('B8').multiply(0.0001),
#         'Red': input.select('B4').multiply(0.0001),
#         'Green': input.select('B3').multiply(0.0001)
#       }).rename('SAVI')
#     evi = image.expression(
#       '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
#       'NIR' : image.select('B8').divide(10000),
#       'RED' : image.select('B4').divide(10000),
#       'BLUE': image.select('B2').divide(10000)}).rename('EVI')
    return image.addBands([ndvi, gndvi, savi])




Google Earth Engine has initialized successfully!


In [246]:
Map1 = geemap.Map(location=[52.244439, 6.843240], zoom=15)
Map1.add_basemap('SATELLITE')
Map1


Map(center=[52.244439, 6.84324], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(c…

In [248]:
fc = ee.FeatureCollection(Map1.draw_last_feature)
region = fc.geometry()
cords = region.getInfo()['coordinates']
cords

[[[6.75627, 52.133732],
  [6.85618, 52.133732],
  [6.85618, 52.170806],
  [6.75627, 52.170806],
  [6.75627, 52.133732]]]

In [249]:
region = ee.Geometry.Polygon([[[6.889473, 52.183777],
  [6.888958, 52.179988],
  [6.869388, 52.179251],
  [6.860805, 52.184619],
  [6.848445, 52.185882],
  [6.841235, 52.173145],
  [6.834025, 52.170724],
  [6.830935, 52.166618],
  [6.834368, 52.157983],
  [6.826313, 52.147085],
  [6.798503, 52.144136],
  [6.789919, 52.13929],
  [6.819789, 52.121586],
  [6.837299, 52.127277],
  [6.837299, 52.147296],
  [6.841763, 52.155301],
  [6.856183, 52.171938],
  [6.880903, 52.160145],
  [6.898756, 52.169622],
  [6.922446, 52.184991],
  [6.945793, 52.194462],
  [6.962273, 52.196356],
  [6.965363, 52.202669],
  [6.935493, 52.209191],
  [6.921073, 52.202458],
  [6.913519, 52.186675],
  [6.889473, 52.183777]]])

In [281]:
myCollection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(region) \
    .filterDate('2019-06-01', '2019-08-31') \
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 1)
listOfImages = myCollection.aggregate_array('system:index').getInfo()
print('Number of images in the collection: ', listOfImages)


Number of images in the collection:  ['20190627T104029_20190627T104030_T31UGT', '20190627T104029_20190627T104030_T32ULC', '20190826T104029_20190826T104232_T31UGT', '20190826T104029_20190826T104232_T31UGU', '20190826T104029_20190826T104232_T32ULC']


In [285]:
visParams = {"min": 0, 
              "max": 1500, 
              "bands": ["B4", "B3", "B2"]}
Images = myCollection.toList(myCollection.size())

ndviParams_diff = {min: 0, max: 1, "palette":  ['00cd00', 'ffff00', 'ffa500', 'cd0000', '8b0000'], "bands": ['NDVI']};

gndviParams_diff = {min: 0, max: 1, "palette":  ['00cd00', 'ffff00', 'ffa500', 'cd0000', '8b0000'], "bands": ['GNDVI']};

saviParams_diff = {min: 0, max: 1, "palette":  ['00cd00', 'ffff00', 'ffa500', 'cd0000', '8b0000'], "bands": ['SAVI']};

ndviParams = {min: -1, max: 1, "palette":  ['white', 'green'], "bands": ['NDVI']};



Map(center=[52.179988, 6.888958], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(…

In [254]:
Image_1 = ee.Image(Images.get(0)).clip(region)
Image_1_metrics = add_metrics(Image_1)

Image_2 = ee.Image(Images.get(3)).clip(region)
Image_2_metrics = add_metrics(Image_2)

Image_diff = (Image_1_metrics.subtract(Image_2_metrics)).abs().select(['NDVI', 'SAVI', 'GNDVI'])

Map3 = geemap.Map(location=[52.179988, 6.888958], zoom=12)
Map3.addLayer(Image_diff, ndviParams_diff, "MedianImage", True) 
Map3

# geemap.ee_export_image(
#     Image_diff, filename='Enschede-surroundings/enschede-diff-20180702-20180806.tif', scale=30, region=region, file_per_band=False
# )


Map(center=[52.1952, 6.773208], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(ch…

In [282]:
Map2 = geemap.Map(location=[ 52.1952, 6.773208], zoom=15)
# Map1.add_basemap('SATELLITE')
Map2
Map2.addLayer(Image_diff, saviParams, "MedianImage", True) 
Map2



https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/videoThumbnails/6f7ccd87b5e4e788fbb6020a566a64b1-2aedafdb4a4fcd4bee4b3e0ece181e10:getPixels


In [263]:
filmArgs = {
    "dimensions": 128,
    "region": region,
    "crs": 'EPSG:3857',
    "min": 0.2, 
    "max" : 1,
    "palette":  ['#640000', '#ff0000', '#ffff00', '#00c800', '#006400'],
#     "palette":  ['#006400', '#00c800', '#ffff00', '#ff0000', '#640000'],
#     "palette": ['white', 'green'],
    "bands": ['NDVI'],
    "framesPerSecond": 1
}

print(myCollection.map(add_metrics).filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 1).getVideoThumbURL(filmArgs));





Map(center=[52.1952, 6.773208], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(ch…

In [264]:
Map4 = geemap.Map(location=[ 52.1952, 6.773208], zoom=15)
# Map1.add_basemap('SATELLITE')
Map4
Map4.addLayer(Image_1_metrics, ndviParams, "MedianImage", True) 
Map4



Map(center=[52.1952, 6.773208], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(ch…

In [ ]:
Map5 = geemap.Map(location=[ 52.1952, 6.773208], zoom=15)
# Map1.add_basemap('SATELLITE')
Map5
Map5.addLayer(Image_2_metrics, ndviParams, "MedianImage", True) 
Map5


